In [ ]:
from farter import ModelConfig

model_config = ModelConfig.load('nvidia_v1')
print(model_config)


In [ ]:
from farter import CandleDataFetcher

model_config.raw_input[0].get_file_path()

data_fetcher = CandleDataFetcher()
raw_data = data_fetcher.fetch_raw_data(model_config)

In [ ]:
from farter import calculate_indicators

indicators = calculate_indicators(model_config, raw_data)


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ta.trend

nvda = raw_data[0][-150:-1]
macd_ref = ta.trend.MACD(nvda.c, window_fast=3, window_slow=8, window_sign=5).macd_diff()

rsi = indicators['MACD-3-8-5'][-150:-1]

fig = make_subplots(rows=2, cols=1, shared_xaxes=True)

fig.add_trace(go.Candlestick(x=nvda.t,
                open=nvda.o,
                high=nvda.h,
                low=nvda.l,
                close=nvda.c), row=2, col=1)

fig.add_trace(go.Scatter(x=nvda.t, y=rsi, name=rsi.name), row=1, col=1)
fig.add_trace(go.Scatter(x=nvda.t, y=macd_ref, name='macd_ref'), row=1, col=1)

fig.show()

In [ ]:
from farter import calculate_target

nvda = raw_data[0]
target = calculate_target(nvda, threshold=0.02)

fig = make_subplots(rows=2, cols=1, shared_xaxes=True)

fig.add_trace(go.Candlestick(x=nvda.t,
                open=nvda.o,
                high=nvda.h,
                low=nvda.l,
                close=nvda.c), row=2, col=1)

fig.add_trace(go.Scatter(x=nvda.t, y=target), row=1, col=1)

fig.show()


In [ ]:
import pandas as pd
import catboost as cb
from sklearn.model_selection import train_test_split

indicators_data = indicators.dropna(axis=0)
target_data = target.dropna(axis=0)

indicators_data = indicators_data[indicators_data.index.isin(target_data.index)]
target_data = target_data[target_data.index.isin(indicators_data.index)]

rows = len(indicators_data.index)
train_size = int(rows * 0.8)

buy_rows = len(target_data[target_data > 0])
sell_rows = len(target_data[target_data < 0])
do_nothing_rows = rows - buy_rows - sell_rows

buy_rate = (1 - buy_rows / rows) / 2
sell_rate = (1 - sell_rows / rows) / 2
do_nothing_rate = (1 - do_nothing_rows / rows) / 2

print(f'buy rate: {buy_rate}, sell rate: {sell_rate}, do nothing rate: {do_nothing_rate}')

weight = pd.Series(index=target_data.index)
weight[target_data > 0] = buy_rate
weight[target_data < 0] = sell_rate
weight[target_data == 0] = do_nothing_rate

classifier = cb.CatBoostClassifier(learning_rate=0.07)
x_train, x_test, y_train, y_test, w_train, w_test = train_test_split(indicators_data, target_data, weight,
                                                                     train_size=train_size)
train_pool = cb.Pool(data=x_train, label=y_train, weight=w_train)
test_pool = cb.Pool(data=x_test, label=y_test)

classifier.fit(X=train_pool, eval_set=test_pool)

predicted = classifier.predict_proba(indicators.tail(150)).transpose()

fig = make_subplots(rows=2, cols=1, shared_xaxes=True)

fig.add_trace(go.Candlestick(x=nvda.tail(150).t,
                open=nvda.tail(150).o,
                high=nvda.tail(150).h,
                low=nvda.tail(150).l,
                close=nvda.tail(150).c), row=2, col=1)

fig.add_trace(go.Scatter(x=nvda.tail(150).t, y=predicted[0], name='Buy'), row=1, col=1)
fig.add_trace(go.Scatter(x=nvda.tail(150).t, y=predicted[1], name='Do Nothing'), row=1, col=1)
fig.add_trace(go.Scatter(x=nvda.tail(150).t, y=predicted[2], name='Sell'), row=1, col=1)

fig.show()

